In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ids = test["PassengerId"]

In [4]:
data.head()


def clean_data(data):
    data = data.drop(["PassengerId", "Ticket", "Name", "Cabin"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)

    data.Embarked.fillna("S", inplace=True)
    return data


data = clean_data(data)
test = clean_data(test)

In [5]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

data["Sex"] = le.fit_transform(data["Sex"])
test["Sex"] = le.transform(test["Sex"])

embarked_mapping = {"S": 1, "C": 2, "Q": 3}
data["Embarked"] = data["Embarked"].map(embarked_mapping)
test["Embarked"] = test["Embarked"].map(embarked_mapping)


data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,1
1,1,1,0,38.0,1,0,71.2833,2
2,1,3,0,26.0,0,0,7.9250,1
3,1,1,0,35.0,1,0,53.1000,1
4,0,3,1,35.0,0,0,8.0500,1


In [7]:
from sklearn.linear_model import LogisticRegression

# from xgboost import XGBClassifier


from sklearn.model_selection import train_test_split


y = data["Survived"]


X = data.drop("Survived", axis=1)


X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.2, random_state=42)

In [13]:
gbc = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

c:\Users\quari\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
preds = gbc.predict(X_val)

from sklearn.metrics import accuracy_score

accuracy_score(y_val, preds)

0.7966339410939691

In [10]:
submission_preds = gbc.predict(test)

In [11]:
df = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission_preds})

In [12]:
df.to_csv("submission.csv", index=False)